In [1]:
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import re

## Dataset creation

In [2]:
# import os
# import librosa
# import soundfile as sf

# # Assuming 'output_filename' is the desired filename for the output WAV file
# output_filename = 'long_form_audio/concatenated_output.wav'
# data=pd.read_csv('../MADHAV_task_1/Task3/metadata_test_clean.tsv', sep='\t', header=None, names=['path', 'text', 'num','sr'])
# # List to store audio chunks
# audio_chunks = []
# target=""
# # Iterate over all files in the directory
# for i in range(10):
#     file_path = f"output/audio_noise{i}.wav"

#     # Load the audio file
#     audio, sr = librosa.load(file_path, sr=None)
#     if(target==""):
#         target=data['text'][i]
#     else:
#         target+=" "+data['text'][i]
#     # Append the audio to the list
#     audio_chunks.append(audio)

# # Concatenate all audio chunks
# concatenated_audio = np.concatenate(audio_chunks)

# # Save the concatenated audio to a new WAV file
# sf.write(output_filename, concatenated_audio, samplerate=sr)
# print(target)

In [3]:
%cd /home/suryansh/MADHAV

/home/suryansh/MADHAV


In [4]:
# txt_file_path = "./hindi_data/givenFolderHSp/txt/12janA.txt"
# file=open("./hindi_data/givenFolderHSp/txt/12janA.txt", 'r')
# target=file.readlines()
# target=re.sub(" +", " ", " ".join(target))
# # Now 'hindi_text' contains the contents of the file as a string
# print(target)

## Model algorithm

In [5]:
def split_audio_using_VAD(input_file, pred, chunk_length_sec=15):
    # Load the audio file
    audio, sr = librosa.load(input_file, sr=None)
    timestamps_df=pd.DataFrame()
    timestamps_df['start']=pred['start'].apply(lambda x: x*sr/1000)
    timestamps_df['end']=pred['end'].apply(lambda x: x*sr/1000)
    print(timestamps_df)
    # Calculate the total duration of the audio in samples
    total_samples = len(audio)

    # Calculate the chunk length in samples
    chunk_length_samples = int(sr * chunk_length_sec)
    print(chunk_length_samples)
    # Create an array to store audio chunks
    audio_chunks = []
    start_sample=0
    end_sample=chunk_length_samples
    split_sec=[]
    k=0
    index=0
    for i, row in timestamps_df.iterrows():
        if (start_sample+chunk_length_samples)>=row['end']:
            end_sample=row['end']
        else:
            audio_chunks.append(audio[int(start_sample):int(end_sample)])
            split_sec.append((index, int(start_sample)/sr, int(end_sample)/sr))
            index+=1
            start_sample=end_sample
            end_sample=row['end']
            while(start_sample+chunk_length_samples<row['end']):
                end_sample=start_sample+chunk_length_samples
                audio_chunks.append(audio[int(start_sample):int(end_sample)])
                split_sec.append((index, int(start_sample)/sr, int(end_sample)/sr))
                start_sample=end_sample
                index+=1
                end_sample=row['end']
            

    last_chunk = audio[int(start_sample):int(total_samples)]
    audio_chunks.append(last_chunk)
    split_sec.append((index, start_sample/sr, total_samples/sr))
    split_sec=pd.DataFrame(split_sec, columns=['index', 'start', 'end'])
    return audio_chunks, split_sec

In [6]:
# from fsmnvad import FSMNVad
# # Specify the input audio file
# input_file = "./hindi_data/givenPMS/newGivenPMS/longAudio/april_v.wav"
# vad = FSMNVad()
# segments = vad.segments_offline(input_file)
# df=pd.DataFrame(segments, columns=['start', 'end'])
# df

In [7]:
# !pip install -U funasr_onnx

In [8]:
# !git lfs install
# !git clone https://huggingface.co/funasr/FSMN-VAD

In [9]:
# from funasr_onnx import Fsmn_vad

# model_dir = "./FSMN-VAD"
# model = Fsmn_vad(model_dir, quantize=True)

# input_file = "./hindi_data/givenFolderHSp/fullAudio/12janA.wav"

# result = model(input_file)

In [10]:
# result = np.asarray(result, dtype=np.int32)
# result=result.reshape((result.shape[1],2))
# df=pd.DataFrame(result, columns=['start', 'end'])
# df

In [11]:
# # Split the audio into 15-second chunks with adjustment for the last chunk
# chunks_array, split_sec = split_audio_using_VAD(input_file, df, 7)

# # Now, chunks_array contains the audio chunks as numpy arrays
# for i, chunk in enumerate(chunks_array):
#     print(f"Chunk {i}: start: {split_sec['start'][i]} end: {split_sec['end'][i]}")

In [12]:
# !pip install -q espnet_model_zoo

In [13]:
# #@title Choose English ASR model { run: "auto" }

# lang = 'en'
# fs = 16000 #@param {type:"integer"}
# tag = 'viks66/asr_train_asr_raw_hindi_bpe500' #@param ["Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave", "kamo-naoyuki/librispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave"] {type:"string"}

In [14]:
# !git lfs install
# !git clone https://huggingface.co/viks66/asr_train_asr_raw_hindi_bpe500

In [15]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


In [16]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    "exp/asr_train_asr_raw_hindi_bpe500/config.yaml",
    "exp/asr_train_asr_raw_hindi_bpe500/valid.acc.ave_10best.pth",
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.5,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [17]:
# import pandas as pd
# import librosa.display
# from IPython.display import display, Audio
# import matplotlib.pyplot as plt
# import time
# import numpy as np
# import soundfile as sf

# col_names = ['path', 'text', 'time', 'sr']
# preds = ""
# target = ""

# for i, chunk in enumerate(chunks_array):
#     start_time = time.time()
#     speech = np.array([])
#     duration=split_sec['end'][i]*1000-split_sec['start'][i] * 1000
#     for _, row in df.iterrows():
#         start_sample = row['start']-split_sec['start'][i] * 1000
#         end_sample = row['end']-split_sec['start'][i] * 1000
# #         print(start_sample, end_sample, duration)
#         if(start_sample<0 and end_sample<0):
#             continue
#         if(start_sample>duration and end_sample>duration):
#             break
# #         print("Y")
#         speech = np.concatenate([speech, chunk[int(max(0,start_sample))*16:int(min(duration, end_sample))*16]])

#     print(len(speech))
#     nbests = speech2text(speech)
#     if len(speech) != 0:
# #         nbests = speech2text(speech)
#         text, *_ = nbests[0]
# #         output_filename = f'results/output_chunk{i}.wav'
# #         sf.write(output_filename, speech, samplerate=16000)
#         if(preds==""):
#             preds=(text_normalizer(text))
#         else:
#             preds += " " + (text_normalizer(text))

#     print(i, "/500")
#     elapsed_time = time.time() - start_time
#     print(f"Time taken: {elapsed_time:.2f} seconds")


In [18]:
predictions=[]
ground_truth=[]

In [20]:
import pandas as pd
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import time
import numpy as np
import soundfile as sf
from funasr_onnx import Fsmn_vad
files=['./hindi_data/givenFolderHSp/fullAudio/12janA.wav', './hindi_data/givenFolderHSp/fullAudio/11janA.wav','./hindi_data/givenFolderPBData/fullAudio/8483_FullAudio.wav','./hindi_data/givenPMS/newGivenPMS/longAudio/april_v.wav' ]
txts=['./hindi_data/givenFolderHSp/txt/12janA.txt', './hindi_data/givenFolderHSp/txt/11janA.txt','./hindi_data/givenFolderPBData/txt/8483_FullAudio.txt','./hindi_data/givenPMS/newGivenPMS/txt/april_v.txt']
# files=['./hindi_data/givenFolderPBData/fullAudio/17162_FullAudio.wav']
# txts=['./hindi_data/givenFolderPBData/txt/17162_FullAudio.txt']
for txt_file_path, input_file in zip(txts, files):
    %cd /home/suryansh/MADHAV
    file=open(txt_file_path, 'r')
    target=""
    target=file.readlines()
    target=re.sub(" +", " ", " ".join(target))
    # Now 'hindi_text' contains the contents of the file as a string
#     print(target)
    ground_truth.append(target)
#     print(ground_truth)
    model_dir = "./FSMN-VAD"
    model = Fsmn_vad(model_dir, quantize=True)
    result = model(input_file)
    result = np.asarray(result, dtype=np.int32)
    result=result.reshape((result.shape[1],2))
    df=pd.DataFrame(result, columns=['start', 'end'])
    # Split the audio into 15-second chunks with adjustment for the last chunk
    chunks_array, split_sec = split_audio_using_VAD(input_file, df, 15)
    %cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

    preds = ""
    for i, chunk in enumerate(chunks_array):
        start_time = time.time()
        speech = np.array([])
        duration=split_sec['end'][i]*1000-split_sec['start'][i] * 1000
        for _, row in df.iterrows():
            start_sample = row['start']-split_sec['start'][i] * 1000
            end_sample = row['end']-split_sec['start'][i] * 1000
    #         print(start_sample, end_sample, duration)
            if(start_sample<0 and end_sample<0):
                continue
            if(start_sample>duration and end_sample>duration):
                break
    #         print("Y")
            speech = np.concatenate([speech, chunk[int(max(0,start_sample))*16:int(min(duration, end_sample))*16]])

        print(len(speech))
    #     nbests = speech2text(chunk)
        if len(speech) != 0:
            nbests = speech2text(speech)
            text, *_ = nbests[0]
    #         output_filename = f'results/output_chunk{i}.wav'
    #         sf.write(output_filename, speech, samplerate=16000)
            if(preds==""):
                preds=(text_normalizer(text))
            else:
                preds += " " + (text_normalizer(text))

        print(i, "/", len(chunks_array))
        elapsed_time = time.time() - start_time
        print(f"Time taken: {elapsed_time:.2f} seconds")
    predictions.append(preds)
    torch.cuda.empty_cache() 

/home/suryansh/MADHAV
        start        end
0     12960.0    58400.0
1     66080.0   628480.0
2    638240.0  1149440.0
3   1153920.0  2094720.0
4   2131200.0  3091360.0
5   3091360.0  3311360.0
6   3322080.0  3880160.0
7   3890080.0  4378080.0
8   4420800.0  4912640.0
9   4960480.0  5438080.0
10  5471680.0  5711040.0
11  5728800.0  6532160.0
12  6548000.0  7115360.0
13  7158400.0  7167680.0
14  7174400.0  7357440.0
15  7361920.0  7710560.0
16  7721600.0  8578560.0
17  8586080.0  8839680.0
18  8844960.0  8981920.0
19  8986400.0  9179200.0
20  9185760.0  9406880.0
240000
/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500
45440
0 / 49
Time taken: 2.60 seconds
232320


/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


1 / 49
Time taken: 14.63 seconds
240000
2 / 49
Time taken: 14.06 seconds
90080
3 / 49
Time taken: 4.93 seconds
230240


KeyboardInterrupt: 

In [ ]:
print(ground_truth)

In [ ]:
print(predictions)

In [23]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions[:-1], ground_truth[:-1]))
wer = WordErrorRate()
print("WER:", wer(predictions[:-1], ground_truth[:-1]))

CER: tensor(0.1550)
WER: tensor(0.3401)


In [24]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions[2], ground_truth[2]))
wer = WordErrorRate()
print("WER:", wer(predictions[2], ground_truth[2]))

CER: tensor(0.1637)
WER: tensor(0.3511)


In [25]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions[3], ground_truth[3]))
wer = WordErrorRate()
print("WER:", wer(predictions[3], ground_truth[3]))

CER: tensor(0.7743)
WER: tensor(0.9241)


In [82]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


In [83]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    "exp/asr_train_asr_raw_hindi_bpe500/config.yaml",
    "exp/asr_train_asr_raw_hindi_bpe500/valid.acc.ave_10best.pth",
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.5,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [84]:
import pandas as pd
import soundfile
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import time

col_names=['path', 'text', 'time', 'sr', 'std']
egs = pd.read_csv("/home/suryansh/MADHAV/hindi_data/givenFolderHSp/tsv/12janA.tsv", sep="\t", names=col_names, header=None, encoding='utf-8', index_col=False)
# print(egs['path'])
preds=[]
target=[]
for index, row in egs.iterrows():
    start_time = time.time()
    speech, rate = soundfile.read("/home/suryansh/MADHAV/hindi_data/givenFolderHSp/finalSplitAudio/" + row["path"])
#     assert fs == int(row["sr"])
    nbests = speech2text(speech)

    text, *_ = nbests[0]
# Uncomment to see details of each prediction
#     print(f"Input Speech: ESPNet_asr_egs/{row['path']}")
    # let us listen to samples
#     display(Audio(speech, rate=rate))
#     librosa.display.waveshow(speech, sr=rate, color="blue")
#     plt.show()
#     print(f"Reference text: {text_normalizer(row['text'])}")
    target.append(text_normalizer(row['text']))
#     print(f"ASR hypothesis: {text_normalizer(text)}")
    preds.append(text_normalizer(text))
    print(index,"/500")
    elapsed_time = time.time() - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")

/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 /500
Time taken: 5.14 seconds
1 /500
Time taken: 5.37 seconds
2 /500
Time taken: 4.41 seconds
3 /500
Time taken: 2.48 seconds
4 /500
Time taken: 4.52 seconds
5 /500
Time taken: 4.49 seconds
6 /500
Time taken: 5.41 seconds
7 /500
Time taken: 5.35 seconds
8 /500
Time taken: 5.75 seconds
9 /500
Time taken: 5.21 seconds
10 /500
Time taken: 4.13 seconds
11 /500
Time taken: 5.15 seconds
12 /500
Time taken: 6.05 seconds
13 /500
Time taken: 5.67 seconds
14 /500
Time taken: 6.45 seconds
15 /500
Time taken: 5.18 seconds
16 /500
Time taken: 5.57 seconds
17 /500
Time taken: 5.97 seconds
18 /500
Time taken: 2.56 seconds
19 /500
Time taken: 5.08 seconds
20 /500
Time taken: 5.53 seconds
21 /500
Time taken: 2.00 seconds
22 /500
Time taken: 4.25 seconds
23 /500
Time taken: 5.51 seconds
24 /500
Time taken: 6.48 seconds
25 /500
Time taken: 5.60 seconds
26 /500
Time taken: 4.99 seconds
27 /500
Time taken: 5.11 seconds
28 /500
Time taken: 5.03 seconds
29 /500
Time taken: 5.10 seconds
30 /500
Time taken: 

In [85]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(preds, target))
wer = WordErrorRate()
print("WER:", wer(preds, target))

CER: tensor(0.1677)
WER: tensor(0.3445)
